# Part 1: Hands-On with LangChain

This notebook demonstrates LangChain's core capabilities using Azure OpenAI. We'll explore basic prompting, sequential chains, and agent-based tool usage.

## Environment Setup

First, let's load our environment variables and configure our Azure OpenAI connection.

In [ ]:
# Environment and configuration setup
import os
import warnings
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

class NotebookConfig:
    """Configuration management for Azure OpenAI in Jupyter notebooks"""
    
    def __init__(self):
        # Load environment variables
        env_path = find_dotenv()
        if env_path:
            load_dotenv(env_path)
            print(f"✅ Loaded environment from: {env_path}")
        else:
            warnings.warn("No .env file found. Using system environment variables only.")
        
        self._load_azure_config()
        self._validate_config()
    
    def _load_azure_config(self):
        """Load Azure OpenAI configuration from environment variables"""
        self.azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')
        self.azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
        self.azure_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'gpt-4o-mini')
        self.azure_api_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-05-01-preview')
        self.tavily_api_key = os.getenv('TAVILY_API_KEY')
    
    def _validate_config(self):
        """Validate critical configuration"""
        errors = []
        
        if not self.azure_api_key:
            errors.append("AZURE_OPENAI_API_KEY is required")
        
        if not self.azure_endpoint:
            errors.append("AZURE_OPENAI_ENDPOINT is required")
            
        if not self.tavily_api_key:
            errors.append("TAVILY_API_KEY is required for search functionality")
        
        if errors:
            raise ValueError(f"Configuration errors: {', '.join(errors)}")
        
        print("✅ All Azure OpenAI configuration validated successfully")
    
    def display_config(self):
        """Display current configuration (hiding secrets)"""
        print("Azure OpenAI Configuration:")
        print(f"  Endpoint: {self.azure_endpoint}")
        print(f"  Deployment: {self.azure_deployment}")
        print(f"  API Version: {self.azure_api_version}")
        print(f"  API Key: {'*' * 20 + self.azure_api_key[-4:] if self.azure_api_key else 'Not set'}")
        print(f"  Tavily API Key: {'*' * 20 + self.tavily_api_key[-4:] if self.tavily_api_key else 'Not set'}")

# Initialize configuration
try:
    config = NotebookConfig()
    config.display_config()
except Exception as e:
    print(f"❌ Failed to load configuration: {e}")
    print("\nPlease ensure you have a .env file with the following variables:")
    print("- AZURE_OPENAI_API_KEY")
    print("- AZURE_OPENAI_ENDPOINT")
    print("- AZURE_OPENAI_CHAT_DEPLOYMENT_NAME (optional, defaults to gpt-4o-mini)")
    print("- TAVILY_API_KEY")
    raise

### 1.1 Basic Prompting and Model I/O with LCEL

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize Azure OpenAI with configuration
llm = AzureChatOpenAI(
    azure_deployment=config.azure_deployment,
    api_version=config.azure_api_version,
    temperature=0.7,
    azure_endpoint=config.azure_endpoint,
    api_key=config.azure_api_key
)

prompt = ChatPromptTemplate.from_template("Question: {question}\nAnswer: Let's think step by step.")
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question": "What is the capital of France?"})
print(response)

### 1.2 Sequential Processing with LCEL (Modern Pattern)

LCEL (LangChain Expression Language) is the modern way to compose operations. Let's build a sequential workflow using the pipe operator.

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Initialize Azure OpenAI with configuration
llm = AzureChatOpenAI(
    azure_deployment=config.azure_deployment,
    api_version=config.azure_api_version,
    temperature=0.7,
    azure_endpoint=config.azure_endpoint,
    api_key=config.azure_api_key
)

# Modern LCEL approach - compose operations with pipe operator
name_prompt = ChatPromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)

catchphrase_prompt = ChatPromptTemplate.from_template(
    "Write a creative catchphrase for the following company: {company_name}"
)

# Build sequential chain using LCEL composition
name_chain = name_prompt | llm | StrOutputParser()

# Create a more complex chain that passes results between steps
def create_sequential_chain():
    """Creates a sequential chain using modern LCEL patterns"""
    
    # Step 1: Generate company name
    step1 = (
        {"product": RunnablePassthrough()} 
        | name_prompt 
        | llm 
        | StrOutputParser()
    )
    
    # Step 2: Generate catchphrase using the company name
    step2 = (
        {"company_name": step1}
        | catchphrase_prompt
        | llm
        | StrOutputParser()
    )
    
    # Combine results into final output
    return {
        "company_name": step1,
        "catchphrase": step2,
        "product": RunnablePassthrough()
    }

# Execute the sequential chain
sequential_chain = create_sequential_chain()
product = "colorful, eco-friendly socks"

print(f"Input: {product}")
print("\\n🔗 Running sequential LCEL chain...")

result = sequential_chain.invoke(product)
print(f"\\n✅ Results:")
print(f"Product: {result['product']}")
print(f"Company Name: {result['company_name']}")
print(f"Catchphrase: {result['catchphrase']}")

### 1.3 Streaming with LCEL

One of LCEL's key advantages is built-in streaming support for real-time responses.

In [ ]:
import time
from langchain_core.callbacks import BaseCallbackHandler

class StreamingCallbackHandler(BaseCallbackHandler):
    """Custom callback handler to demonstrate streaming"""
    
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(token, end="", flush=True)

# Create a streaming chain
streaming_prompt = ChatPromptTemplate.from_template(
    "Write a brief story about {topic}. Make it engaging and creative."
)

streaming_chain = (
    streaming_prompt 
    | llm.with_config({"callbacks": [StreamingCallbackHandler()]})
    | StrOutputParser()
)

print("🌊 Streaming response for a story about 'space exploration':")
print("=" * 50)

# Note: In a real notebook, you'd see the text appear token by token
response = streaming_chain.invoke({"topic": "space exploration"})

print("\n" + "=" * 50)
print("✅ Streaming complete!")

### 1.4 Function Calling and Basic Agents

LangChain agents can use tools (functions) to interact with external systems. This demonstrates basic agent capabilities before we explore multi-agent systems in a dedicated notebook.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor

# Initialize Azure OpenAI for agents (temperature=0 for more deterministic responses)
llm_agent = AzureChatOpenAI(
    azure_deployment=config.azure_deployment,
    api_version=config.azure_api_version,
    temperature=0,
    azure_endpoint=config.azure_endpoint,
    api_key=config.azure_api_key
)

# Define tools that the agent can use
search_tool = TavilySearchResults(
    max_results=2,
    api_key=config.tavily_api_key,
    description="Search the web for current information"
)

@tool
def calculate(expression: str) -> str:
    """A simple calculator that evaluates basic mathematical expressions.
    
    Args:
        expression: A mathematical expression like '2+2' or '4**0.5'
    """
    try:
        # Basic safety check - only allow simple math operations
        allowed_chars = set('0123456789+-*/().**')
        if not all(c in allowed_chars or c.isspace() for c in expression):
            return "Error: Only basic mathematical operations are allowed"
        
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error evaluating expression: {e}"

@tool 
def format_text(text: str, style: str = "upper") -> str:
    """Format text in different styles.
    
    Args:
        text: The text to format
        style: The formatting style ('upper', 'lower', 'title')
    """
    if style == "upper":
        return text.upper()
    elif style == "lower":
        return text.lower()
    elif style == "title":
        return text.title()
    else:
        return f"Unknown style: {style}. Use 'upper', 'lower', or 'title'"

# Create agent with tools
tools = [search_tool, calculate, format_text]

# Pull a pre-built prompt for function calling agents
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm_agent, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=3)

print("🤖 Testing agent with multiple tool usage...")
print("=" * 50)

# Test multi-step reasoning with tool usage
response = agent_executor.invoke({
    "input": "Calculate 16 raised to the power of 0.5, then format the result as 'The answer is X' in title case"
})

print(f"\\n✅ Final Response: {response['output']}")
print("=" * 50)
print("\\n📝 Note: This demonstrates single-agent tool usage. For complex multi-agent workflows, see the LangGraph notebook.")

## Summary: LangChain Fundamentals

This notebook covered the core concepts of LangChain using modern patterns:

### Key Concepts Learned:
1. **LCEL (LangChain Expression Language)**: Modern composition using pipe operators
2. **Sequential Processing**: Chaining operations with result passing between steps
3. **Streaming**: Real-time token streaming for better user experience  
4. **Function Calling**: Single agents using tools for external capabilities

### LangChain's Philosophy:
- **"Compose Everything"**: Build complex workflows by composing simple components
- **LCEL as Core**: Unified interface for chaining, streaming, and async operations
- **Tool Integration**: Rich ecosystem of 700+ integrations available

### Next Steps:
- **Multi-Agent Systems**: See `3_langchain_agents_langgraph.ipynb` for complex agent workflows
- **Production Patterns**: Error handling, retry logic, and monitoring
- **Advanced LCEL**: Complex routing, fallbacks, and parallel execution

### Production Considerations:
- Always use environment variables for API keys
- Implement proper error handling and timeout management
- Add logging and monitoring for production deployments
- Consider token usage and cost optimization strategies